# 텍스트 전처리
* 불용어 처리: kss 문장분리, 중복 제거, hanspell 맞춤법 교정=> 'sentence' 칼럼
* 토큰 통일 필요: 정규화(Normalization), 표제어 추출(Lemmatization)
* 토크나이저: soynlp, kkma => 'word ' 칼럼
* 대상 품사: NNP(고유 명사), NNG(일반 명사) + VV(동사), VA(형용사)

### 0. 데이터, 라이브러리 Import

In [ ]:
!pip3 install kss
!pip install git+https://github.com/ssut/py-hanspell.git

In [ ]:
!pip3 install soynlp
!pip3 install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 416.8/416.8 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.3/465.3 kB 44.3 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import kss
from soynlp.normalizer import repeat_normalize
from konlpy.tag import Kkma

import re

from tqdm import tqdm

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/BERT_modeling/raw_data.csv')
data.head()

,id,rank,company,name1,name2,category,volume,alc,type,price,review_num,click,link,score,real_date,buy,real_review,split_sentences,sentence
0,1,1,배상면주가,느린마을 막걸리,느린마을 막걸리 6도,막걸리/탁주,750,6.0,페트병,3600,33543,162,https://cr.shopping.naver.com/adcr.nhn?x=c717o...,1,21.02.21.,인터파크쇼핑,상한겁니까? 아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까? 처음 도착했을때 택...,"['상한겁니까?', '아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까?', '처음...",상한겁니까
1,1,1,배상면주가,느린마을 막걸리,느린마을 막걸리 6도,막걸리/탁주,750,6.0,페트병,3600,33543,162,https://cr.shopping.naver.com/adcr.nhn?x=c717o...,1,21.02.21.,인터파크쇼핑,상한겁니까? 아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까? 처음 도착했을때 택...,"['상한겁니까?', '아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까?', '처음...",아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까
2,1,1,배상면주가,느린마을 막걸리,느린마을 막걸리 6도,막걸리/탁주,750,6.0,페트병,3600,33543,162,https://cr.shopping.naver.com/adcr.nhn?x=c717o...,1,21.02.21.,인터파크쇼핑,상한겁니까? 아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까? 처음 도착했을때 택...,"['상한겁니까?', '아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까?', '처음...",처음 도착했을때 택배상태 보고 경악했습니다
3,1,1,배상면주가,느린마을 막걸리,느린마을 막걸리 6도,막걸리/탁주,750,6.0,페트병,3600,33543,162,https://cr.shopping.naver.com/adcr.nhn?x=c717o...,1,21.02.21.,인터파크쇼핑,상한겁니까? 아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까? 처음 도착했을때 택...,"['상한겁니까?', '아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까?', '처음...",이게 뭐죠
4,1,1,배상면주가,느린마을 막걸리,느린마을 막걸리 6도,막걸리/탁주,750,6.0,페트병,3600,33543,162,https://cr.shopping.naver.com/adcr.nhn?x=c717o...,1,21.02.21.,인터파크쇼핑,상한겁니까? 아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까? 처음 도착했을때 택...,"['상한겁니까?', '아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까?', '처음...",아파트단지 분리수거함을 봐도 이거보다 150배는 나은데요


### 1. kss - Sentence Split

In [ ]:
paragraph = data['real_review']
filtered_sentences = []
data['split_sentences']=''

In [ ]:
index=0
for i in tqdm(paragraph, desc='Processing sentences'):
  sentences = kss.split_sentences(i)
  data['split_sentences'][index] = sentences
  index +=1

### 2. hanspell - Spelling Check

In [ ]:
#combined_df로 저장해~
df1 = pd.read_csv('/content/drive/MyDrive/BERT_modeling/you_df4.csv')
temp_ = re.sub(r"[^ㄱ-ㅎ가-힣\s]", "", temp_)

for i in tqdm(range(len(df1))):
  temp_ = df1['sentence'][i]
  temp_ = re.sub(r"[^ㄱ-ㅎ가-힣\s]", "", temp_)
  temp2_ = spell_checker.check(temp_)
  a_ = temp2_.as_dict()
  b_ = a_['checked']
  df1['sentence'][i] = b_


  0%|          | 0/76409 [00:00<?, ?it/s]<ipython-input-87-7d61691d4b0f>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1['sentence'][i] = b_
100%|██████████| 76409/76409 [2:35:12<00:00,  8.21it/s]


In [ ]:
pd.to_csv('/content/drive/MyDrive/BERT_modeling/hanspell_df.csv')

### 3. soynlp - Noramlization

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/BERT_modeling/hanspell_df.csv')
df.head()

,id,rank,company,name1,name2,category,volume,alc,type,price,review_num,click,link,score,real_date,buy,real_review,split_sentences,sentence
0,1,1,배상면주가,느린마을 막걸리,느린마을 막걸리 6도,막걸리/탁주,750,6.0,페트병,3600,33543,162,https://cr.shopping.naver.com/adcr.nhn?x=c717o...,1,21.02.21.,인터파크쇼핑,상한겁니까? 아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까? 처음 도착했을때 택...,"['상한겁니까?', '아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까?', '처음...",상한 겁니까
1,1,1,배상면주가,느린마을 막걸리,느린마을 막걸리 6도,막걸리/탁주,750,6.0,페트병,3600,33543,162,https://cr.shopping.naver.com/adcr.nhn?x=c717o...,1,21.02.21.,인터파크쇼핑,상한겁니까? 아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까? 처음 도착했을때 택...,"['상한겁니까?', '아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까?', '처음...",구매확정합니다
2,1,1,배상면주가,느린마을 막걸리,느린마을 막걸리 6도,막걸리/탁주,750,6.0,페트병,3600,33543,162,https://cr.shopping.naver.com/adcr.nhn?x=c717o...,1,21.02.21.,인터파크쇼핑,상한겁니까? 아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까? 처음 도착했을때 택...,"['상한겁니까?', '아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까?', '처음...",올바른 상품이 왔더니 이건 도대체 다시는 여기 거 안 먹습니다
3,1,1,배상면주가,느린마을 막걸리,느린마을 막걸리 6도,막걸리/탁주,750,6.0,페트병,3600,33543,162,https://cr.shopping.naver.com/adcr.nhn?x=c717o...,1,21.02.21.,인터파크쇼핑,상한겁니까? 아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까? 처음 도착했을때 택...,"['상한겁니까?', '아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까?', '처음...",올바른 상태
4,1,1,배상면주가,느린마을 막걸리,느린마을 막걸리 6도,막걸리/탁주,750,6.0,페트병,3600,33543,162,https://cr.shopping.naver.com/adcr.nhn?x=c717o...,1,21.02.21.,인터파크쇼핑,상한겁니까? 아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까? 처음 도착했을때 택...,"['상한겁니까?', '아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까?', '처음...",저번에 시킨 박스는 제품 상하 뒤집지 말아달란 스티커와 올바른 박스


In [ ]:
# 결측값 drop
df = df.dropna(subset=['sentence'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 305313 entries, 0 to 305638
Data columns (total 19 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   id               305313 non-null  int64  
 1   rank             305313 non-null  int64  
 2   company          305313 non-null  object 
 3   name1            305313 non-null  object 
 4   name2            305313 non-null  object 
 5   category         305313 non-null  object 
 6   volume           305313 non-null  int64  
 7   alc              305313 non-null  float64
 8   type             305313 non-null  object 
 9   price            305313 non-null  int64  
 10  review_num       305313 non-null  int64  
 11  click            305313 non-null  int64  
 12  link             305313 non-null  object 
 13  score            305313 non-null  int64  
 14  real_date        305313 non-null  object 
 15  buy              305313 non-null  object 
 16  real_review      305313 non-null  obje

In [ ]:
# 정규화 진행(soynlp)
def normalization(sentence):
  normalized_sentence = repeat_normalize(sentence)
  return normalized_sentence

df['normalized'] = df['sentence'].apply(normalization)

### 4. kkma - Lemmatization, Pos Tagging

In [ ]:
# 표제어 추출(kkma) -> 1시간 소요
def lemmatization(sentence):
    tokenizer = Kkma()
    tokens = tokenizer.pos(sentence)
    lemmatized_words = []
    for token in tokens:
        word, pos_tag = token
        if pos_tag.startswith('NNP') or pos_tag.startswith('NNG'):
            lemmatized_words.append(word)
        elif pos_tag.startswith('VA') or pos_tag.startswith('VV'):
            lemmatized_words.append(word)
    return ' '.join(lemmatized_words)

df['words'] = ''
with tqdm(total=len(df)) as pbar:
    for idx, row in df.iterrows():
        df.at[idx, 'words'] = lemmatization(row['normalized'])
        pbar.update(1)

df.head()

100%|██████████| 305313/305313 [1:02:00<00:00, 82.06it/s] 


,id,rank,company,name1,name2,category,volume,alc,type,price,...,click,link,score,real_date,buy,real_review,split_sentences,sentence,normalized,words
0,1,1,배상면주가,느린마을 막걸리,느린마을 막걸리 6도,막걸리/탁주,750,6.0,페트병,3600,...,162,https://cr.shopping.naver.com/adcr.nhn?x=c717o...,1,21.02.21.,인터파크쇼핑,상한겁니까? 아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까? 처음 도착했을때 택...,"['상한겁니까?', '아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까?', '처음...",상한 겁니까,상한 겁니까,상한 걸
1,1,1,배상면주가,느린마을 막걸리,느린마을 막걸리 6도,막걸리/탁주,750,6.0,페트병,3600,...,162,https://cr.shopping.naver.com/adcr.nhn?x=c717o...,1,21.02.21.,인터파크쇼핑,상한겁니까? 아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까? 처음 도착했을때 택...,"['상한겁니까?', '아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까?', '처음...",구매확정합니다,구매확정합니다,구매 확정
2,1,1,배상면주가,느린마을 막걸리,느린마을 막걸리 6도,막걸리/탁주,750,6.0,페트병,3600,...,162,https://cr.shopping.naver.com/adcr.nhn?x=c717o...,1,21.02.21.,인터파크쇼핑,상한겁니까? 아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까? 처음 도착했을때 택...,"['상한겁니까?', '아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까?', '처음...",올바른 상품이 왔더니 이건 도대체 다시는 여기 거 안 먹습니다,올바른 상품이 왔더니 이건 도대체 다시는 여기 거 안 먹습니다,올바르 상품 오 그 먹
3,1,1,배상면주가,느린마을 막걸리,느린마을 막걸리 6도,막걸리/탁주,750,6.0,페트병,3600,...,162,https://cr.shopping.naver.com/adcr.nhn?x=c717o...,1,21.02.21.,인터파크쇼핑,상한겁니까? 아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까? 처음 도착했을때 택...,"['상한겁니까?', '아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까?', '처음...",올바른 상태,올바른 상태,올바르 상태
4,1,1,배상면주가,느린마을 막걸리,느린마을 막걸리 6도,막걸리/탁주,750,6.0,페트병,3600,...,162,https://cr.shopping.naver.com/adcr.nhn?x=c717o...,1,21.02.21.,인터파크쇼핑,상한겁니까? 아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까? 처음 도착했을때 택...,"['상한겁니까?', '아니면 그래프 보듯이 신맛이 끝까지 간 상태입니까?', '처음...",저번에 시킨 박스는 제품 상하 뒤집지 말아달란 스티커와 올바른 박스,저번에 시킨 박스는 제품 상하 뒤집지 말아달란 스티커와 올바른 박스,저번 시키 박스 제품 상하 뒤집 말 스티커 올바르 박스


In [ ]:
# 품사태깅 이뤄지지 않은 결측값 drop
df = df.dropna(subset=['words'])
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 305313 entries, 0 to 305638
Data columns (total 21 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   id               305313 non-null  int64  
 1   rank             305313 non-null  int64  
 2   company          305313 non-null  object 
 3   name1            305313 non-null  object 
 4   name2            305313 non-null  object 
 5   category         305313 non-null  object 
 6   volume           305313 non-null  int64  
 7   alc              305313 non-null  float64
 8   type             305313 non-null  object 
 9   price            305313 non-null  int64  
 10  review_num       305313 non-null  int64  
 11  click            305313 non-null  int64  
 12  link             305313 non-null  object 
 13  score            305313 non-null  int64  
 14  real_date        305313 non-null  object 
 15  buy              305313 non-null  object 
 16  real_review      305313 non-null  obje

In [ ]:
df_pos = df.copy()
df_pos.to_csv('/content/drive/MyDrive/BERT_modeling/pos_dataframe.csv')